# Machine Learning Workflow for Predictive Safety Risk Classifier

This notebook implements an end-to-end ML workflow for predicting high-risk safety zones in Chicago. We:
- Load the engineered datasets (from the Feature Engineering step)
- Establish a baseline with Logistic Regression
- Iterate over candidate models (Random Forest and XGBoost)
- Tune the best-performing model (XGBoost) via GridSearchCV and threshold selection
- Evaluate on a held-out test set and save the final model for deployment

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import joblib

# Step 1: Load engineered datasets
train_df = pd.read_csv("../train_engineered.csv").dropna()
val_df = pd.read_csv("../val_engineered.csv").dropna()
test_df = pd.read_csv("../test_engineered.csv").dropna()

# Step 2: Split features and target (dropping 'CrimeDensity' since it is an engineered feature we won't use here)
X_train = train_df.drop(columns=["Risk", "CrimeDensity"])
y_train = train_df["Risk"]
X_val = val_df.drop(columns=["Risk", "CrimeDensity"])
y_val = val_df["Risk"]
X_test = test_df.drop(columns=["Risk", "CrimeDensity"])
y_test = test_df["Risk"]

# Balance the training set using SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
print("Balanced y_train distribution:")
print(y_train_bal.value_counts(normalize=True))
print(f"Train shape: {X_train_bal.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")

Balanced y_train distribution:
Risk
0    0.5
1    0.5
Name: proportion, dtype: float64
Train shape: (38724, 6), Val shape: (5165, 6), Test shape: (5165, 6)


## Step 1: Baseline Model

We begin with a simple Logistic Regression model as our baseline. Note the extreme class weight used to force the model to learn from the minority class. We then examine the probability distribution and set a low threshold to capture nearly all positives.

In [2]:
# Baseline: Logistic Regression
lr_model = LogisticRegression(max_iter=1000, random_state=42, C=0.1, class_weight={0: 1, 1: 50000})
print("Training Logistic Regression on balanced data with", X_train_bal.shape[0], "rows.")
print("Balanced y_train distribution:\n", pd.Series(y_train_bal).value_counts(normalize=True))

lr_model.fit(X_train_bal, y_train_bal)
y_val_prob_lr = lr_model.predict_proba(X_val)[:, 1]

print("Logistic Regression probability range:", y_val_prob_lr.min(), "-", y_val_prob_lr.max())
print("First 20 predicted probabilities:", y_val_prob_lr[:20])

# Using an extremely low threshold to force positive predictions (for baseline comparison)
baseline_thresh = 0.000005
y_val_pred_lr = (y_val_prob_lr >= baseline_thresh).astype(int)

print("\nLogistic Regression - Validation Set Performance (threshold {:.6f}):".format(baseline_thresh))
print("Predicted distribution:\n", pd.Series(y_val_pred_lr).value_counts(normalize=True))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred_lr))
print(classification_report(y_val, y_val_pred_lr, zero_division=0))
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_lr):.3f}")

Training Logistic Regression on balanced data with 38724 rows.
Balanced y_train distribution:
 Risk
0    0.5
1    0.5
Name: proportion, dtype: float64
Logistic Regression probability range: 0.9999792692244465 - 0.9999933979059933
First 20 predicted probabilities: [0.99997943 0.99997943 0.9999797  0.99997962 0.99998026 0.99998708
 0.99997955 0.99997976 0.99997998 0.99997968 0.99998004 0.99997958
 0.99997971 0.99997972 0.99997984 0.99997948 0.99997998 0.99998006
 0.99997953 0.99997958]

Logistic Regression - Validation Set Performance (threshold 0.000005):
Predicted distribution:
 1    1.0
Name: proportion, dtype: float64
Confusion Matrix:
 [[   0 4149]
 [   0 1016]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4149
           1       0.20      1.00      0.33      1016

    accuracy                           0.20      5165
   macro avg       0.10      0.50      0.16      5165
weighted avg       0.04      0.20      0.06      5165


## Step 2: Model Iteration

We now test two candidate models: Random Forest and XGBoost. Our goal is to compare performance (in terms of recall, accuracy, and predicted positives) against the baseline.

In [3]:
# Candidate Model 1: Random Forest
rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    random_state=42,
    class_weight={0: 1, 1: 100000}
)
print("Training Random Forest...")
rf_model.fit(X_train_bal, y_train_bal)
y_val_prob_rf = rf_model.predict_proba(X_val)[:, 1]

print("Random Forest probability range:", y_val_prob_rf.min(), "-", y_val_prob_rf.max())
y_val_pred_rf = (y_val_prob_rf >= 0.00005).astype(int)
print("\nRandom Forest - Validation Set Performance (threshold 0.00005):")
print("Predicted distribution:\n", pd.Series(y_val_pred_rf).value_counts(normalize=True))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred_rf))
print(classification_report(y_val, y_val_pred_rf, zero_division=0))
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_rf):.3f}")

# Candidate Model 2: XGBoost
xgb_model = xgb.XGBClassifier(
    eval_metric="logloss",
    random_state=42,
    scale_pos_weight=10000,
    max_depth=15,
    learning_rate=0.3,
    min_child_weight=1
)
print("\nTraining XGBoost...")
xgb_model.fit(X_train_bal, y_train_bal)
y_val_prob_xgb = xgb_model.predict_proba(X_val)[:, 1]

print("XGBoost probability range:", y_val_prob_xgb.min(), "-", y_val_prob_xgb.max())
y_val_pred_xgb = (y_val_prob_xgb >= 0.005).astype(int)
print("\nXGBoost - Validation Set Performance (threshold 0.005):")
print("Predicted distribution:\n", pd.Series(y_val_pred_xgb).value_counts(normalize=True))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred_xgb))
print(classification_report(y_val, y_val_pred_xgb, zero_division=0))
print(f"Accuracy: {accuracy_score(y_val, y_val_pred_xgb):.3f}")

Training Random Forest...
Random Forest probability range: 0.0 - 1.0

Random Forest - Validation Set Performance (threshold 0.00005):
Predicted distribution:
 0    0.801355
1    0.198645
Name: proportion, dtype: float64
Confusion Matrix:
 [[4139   10]
 [   0 1016]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4149
           1       0.99      1.00      1.00      1016

    accuracy                           1.00      5165
   macro avg       1.00      1.00      1.00      5165
weighted avg       1.00      1.00      1.00      5165

Accuracy: 0.998

Training XGBoost...
XGBoost probability range: 4.4638273e-05 - 1.0

XGBoost - Validation Set Performance (threshold 0.005):
Predicted distribution:
 0    0.803291
1    0.196709
Name: proportion, dtype: float64
Confusion Matrix:
 [[4149    0]
 [   0 1016]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4149
           1       1.00  

## Step 3: Hyperparameter Tuning of XGBoost

Based on the iteration results, XGBoost shows a promising balance of recall and accuracy. We now tune its hyperparameters using GridSearchCV on the balanced and scaled training data. We also perform a threshold sweep on the validation set to fine-tune the predicted positives.

In [4]:
from sklearn.preprocessing import StandardScaler

# Scale the training, validation, and test sets
scaler = StandardScaler()
X_train_bal_scaled = scaler.fit_transform(X_train_bal)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define parameter grid for XGBoost
param_grid = {
    'max_depth': [10, 15, 20],
    'learning_rate': [0.1, 0.3, 0.5],
    'n_estimators': [100, 500, 1000],
    'min_child_weight': [1, 3],
    'scale_pos_weight': [10000, 15000]
}

xgb_estimator = xgb.XGBClassifier(eval_metric="logloss", random_state=42)

grid_search = GridSearchCV(estimator=xgb_estimator, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train_bal_scaled, y_train_bal)

print("Best XGBoost parameters:", grid_search.best_params_)
best_xgb = grid_search.best_estimator_

# Perform threshold tuning on the validation set
y_val_prob_xgb_tuned = best_xgb.predict_proba(X_val_scaled)[:, 1]
thresholds = [0.001, 0.002, 0.003, 0.004, 0.005]

print("\nThreshold Tuning on Validation Set:")
for thresh in thresholds:
    y_val_pred = (y_val_prob_xgb_tuned >= thresh).astype(int)
    pos_count = y_val_pred.sum()
    print(f"\nThreshold: {thresh}")
    print("Predicted positives:", pos_count, "out of", len(y_val_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))
    print(classification_report(y_val, y_val_pred, zero_division=0))
    print(f"Accuracy: {accuracy_score(y_val, y_val_pred):.3f}")

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best XGBoost parameters: {'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100, 'scale_pos_weight': 10000}

Threshold Tuning on Validation Set:

Threshold: 0.001
Predicted positives: 1016 out of 5165
Confusion Matrix:
 [[4149    0]
 [   0 1016]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4149
           1       1.00      1.00      1.00      1016

    accuracy                           1.00      5165
   macro avg       1.00      1.00      1.00      5165
weighted avg       1.00      1.00      1.00      5165

Accuracy: 1.000

Threshold: 0.002
Predicted positives: 1016 out of 5165
Confusion Matrix:
 [[4149    0]
 [   0 1016]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4149
           1       1.00      1.00      1.00      1016

    accuracy                           1.00      5165

## Step 4: Final Evaluation on the Test Set

Using the tuned XGBoost model (and an appropriate threshold determined from the validation set, e.g., 0.003), we evaluate the final model on the held-out test set.

In [5]:
# Select the best threshold based on validation (adjust as needed)
best_thresh = 0.003

y_test_prob = best_xgb.predict_proba(X_test_scaled)[:, 1]
y_test_pred = (y_test_prob >= best_thresh).astype(int)

print("Final XGBoost - Test Set Performance (threshold {:.3f}):".format(best_thresh))
print("Predicted distribution:\n", pd.Series(y_test_pred).value_counts(normalize=True))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, zero_division=0))
print(f"Test Set Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")

Final XGBoost - Test Set Performance (threshold 0.003):
Predicted distribution:
 0    0.803485
1    0.196515
Name: proportion, dtype: float64
Confusion Matrix:
 [[4150    0]
 [   0 1015]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4150
           1       1.00      1.00      1.00      1015

    accuracy                           1.00      5165
   macro avg       1.00      1.00      1.00      5165
weighted avg       1.00      1.00      1.00      5165

Test Set Accuracy: 1.000


## Step 5: Save the Best Model

We now save the tuned XGBoost model (along with the scaler) for future use or deployment.

In [6]:
# Save the best XGBoost model and the scaler for inference
joblib.dump(best_xgb, "../best_model.pk1")
joblib.dump(scaler, "../scaler.pkl")
print("Best XGBoost model saved as 'best_model.pk1'")
print("Scaler saved as 'scaler.pkl'")

Best XGBoost model saved as 'best_model.pk1'
Scaler saved as 'scaler.pkl'
